In [1]:
import pickle
import statistics
import numpy as np
import pandas as pd
from datetime import datetime
import preprocessing_functions

Load Fitbit

In [2]:
fitbit = pd.read_pickle('./data/fitbit_to_baseline_model.pkl')
fitbit["date"] = pd.to_datetime(pd.to_datetime(fitbit["date"]).dt.date)
fitbit

id       date  nightly_temperature  nremhr  \
0     621e2e8e67b776a24055b564 2021-05-24            34.137687  57.432   
1     621e301e67b776a240608a72 2021-05-24            34.710232     NaN   
3     621e301367b776a24057738e 2021-05-24                  NaN     NaN   
4     621e360b67b776a24039709f 2021-05-24                  NaN  68.321   
5     621e2fce67b776a240279baa 2021-05-24            34.783747  83.170   
...                        ...        ...                  ...     ...   
4959  621e345267b776a240691064 2022-01-17                  NaN     NaN   
4960  621e310d67b776a24003096d 2022-01-17                  NaN     NaN   
4961  621e324e67b776a2400191cb 2022-01-17            33.104768     NaN   
4962  621e30f467b776a240f22944 2022-01-17            34.352490     NaN   
4963  621e2f1b67b776a240b3d87c 2022-01-17            34.404580  65.782   

       rmssd  spo2  full_sleep_breathing_rate  stress_score  \
0     89.603   NaN                       14.8          78.0   
1        NaN   NaN                        NaN          74.0   
3        NaN   NaN                        NaN           NaN   
4     31.413   NaN                       15.4          69.0   
5     15.941  96.6                       15.2          70.0   
...      ...   ...                        ...           ...   
4959     NaN   NaN                        NaN           NaN   
4960     NaN   NaN                        NaN           NaN   
4961     NaN   NaN                        NaN           NaN   
4962     NaN   NaN                       13.4           NaN   
4963  23.793   NaN                       16.0          72.0   

      sleep_points_percentage  exertion_points_percentage  ...  Martial Arts  \
0                    0.833333                       0.675  ...           0.0   
1                    0.700000                       0.700  ...           NaN   
3                         NaN                         NaN  ...           0.0   
4                    0.633333                       0.650  ...           NaN   
5                    0.733333                       0.725  ...           NaN   
...                       ...                         ...  ...           ...   
4959                      NaN                         NaN  ...           NaN   
4960                      NaN                         NaN  ...           NaN   
4961                      NaN                         NaN  ...           NaN   
4962                      NaN                         NaN  ...           NaN   
4963                 0.766667                       0.725  ...           NaN   

      Run  Spinning  Sport  Swim  Treadmill  Walk  Weights  Workout  \
0     0.0       0.0    0.0   0.0        0.0   1.0      0.0      0.0   
1     NaN       NaN    NaN   NaN        NaN   NaN      NaN      NaN   
3     1.0       0.0    0.0   0.0        0.0   0.0      0.0      0.0   
4     NaN       NaN    NaN   NaN        NaN   NaN      NaN      NaN   
5     NaN       NaN    NaN   NaN        NaN   NaN      NaN      NaN   
...   ...       ...    ...   ...        ...   ...      ...      ...   
4959  NaN       NaN    NaN   NaN        NaN   NaN      NaN      NaN   
4960  NaN       NaN    NaN   NaN        NaN   NaN      NaN      NaN   
4961  NaN       NaN    NaN   NaN        NaN   NaN      NaN      NaN   
4962  NaN       NaN    NaN   NaN        NaN   NaN      NaN      NaN   
4963  NaN       NaN    NaN   NaN        NaN   NaN      NaN      NaN   

      Yoga/Pilates  
0              0.0  
1              NaN  
3              0.0  
4              NaN  
5              NaN  
...            ...  
4959           NaN  
4960           NaN  
4961           NaN  
4962           NaN  
4963           NaN  

[4907 rows x 64 columns]

Load TTM

In [3]:
ttm = pd.read_pickle('./data/surveys/ttm.pkl')
ttm = ttm.rename(columns={"user_id": "id", "submitdate": "date"})
ttm["date"] = pd.to_datetime(pd.to_datetime(ttm["date"]).dt.date)
ttm.drop_duplicates(subset='id', keep='first', inplace=True)
ttm = ttm.drop(columns=['type'])
ttm

id       date             stage  \
0  621e2e8e67b776a24055b564 2021-07-26       Maintenance   
0  621e2eaf67b776a2406b14ac 2021-11-29     Contemplation   
0  621e2ed667b776a24085d8d1 2021-06-07       Preparation   
0  621e2f3967b776a240c654db 2021-05-31       Preparation   
0  621e2f6167b776a240e082a9 2021-06-01     Contemplation   
0  621e2f7a67b776a240f14425 2021-05-31       Maintenance   
0  621e2f9167b776a240011ccb 2021-11-28  Precontemplation   
0  621e2fb367b776a24015accd 2021-05-31     Contemplation   
0  621e2fce67b776a240279baa 2021-05-31     Contemplation   
0  621e2ff067b776a2403eb737 2021-11-29            Action   
0  621e300767b776a2404dc717 2022-01-17            Action   
0  621e301367b776a24057738e 2021-07-26       Maintenance   
2  621e301e67b776a240608a72 2021-08-28            Action   
0  621e30c867b776a240d4aa6c 2021-06-04       Preparation   
0  621e30e267b776a240e5bf90 2021-11-23  Precontemplation   
0  621e30e467b776a240e817c7 2021-08-01     Contemplation   
0  621e30f467b776a240f22944 2022-01-18     Contemplation   
0  621e310d67b776a24003096d 2022-01-24       Preparation   
0  621e312a67b776a240164d59 2022-01-17     Contemplation   
0  621e314867b776a24029ebf9 2021-05-31       Preparation   
0  621e323667b776a240f19134 2021-11-23            Action   
0  621e324e67b776a2400191cb 2021-11-27  Precontemplation   
0  621e326767b776a24012e179 2021-06-07       Maintenance   
0  621e328667b776a240281372 2021-05-31       Maintenance   
0  621e329067b776a2402ffad2 2021-07-26       Maintenance   
0  621e32af67b776a24045b4cf 2021-07-26       Maintenance   
0  621e32d067b776a2405b7d54 2021-12-09       Preparation   
0  621e32d967b776a240627414 2021-05-31       Maintenance   
0  621e32e667b776a2406d2f1c 2021-11-23     Contemplation   
0  621e331067b776a24085dd3f 2021-11-23  Precontemplation   
0  621e332267b776a24092a584 2021-06-02       Maintenance   
0  621e333567b776a240a0c217 2021-11-23     Contemplation   
0  621e333967b776a240a3cd06 2021-11-27       Maintenance   
0  621e335a67b776a240bb12ff 2021-05-31            Action   
0  621e337667b776a240ce78ab 2021-06-07     Contemplation   
0  621e339967b776a240e502de 2021-11-22       Maintenance   
1  621e33b067b776a240f39e56 2021-07-26     Contemplation   
0  621e33cf67b776a240087de9 2021-11-24       Maintenance   
0  621e33ed67b776a2401cf5f7 2021-06-03       Maintenance   
0  621e341067b776a24037b105 2021-05-31     Contemplation   
0  621e345c67b776a2407146a8 2021-08-17     Contemplation   
0  621e346f67b776a24081744f 2021-11-24     Contemplation   
0  621e34db67b776a240c9c2be 2021-06-09       Preparation   
0  621e34ec67b776a240d60873 2021-06-02       Maintenance   
1  621e34f767b776a240de4e1a 2021-06-07  Precontemplation   
0  621e351a67b776a240f6204b 2021-07-26            Action   
0  621e356967b776a24027bd9f 2021-11-24       Maintenance   
0  621e362467b776a2404ad513 2021-06-02     Contemplation   
0  621e366567b776a24076a727 2021-11-22       Maintenance   
0  621e367e67b776a24087d75d 2021-11-24  Precontemplation   
0  621e36c267b776a240ba2756 2021-05-31            Action   
0  621e36f967b776a240e5e7c9 2021-05-31            Action   
0  621e375b67b776a240290cdc 2021-06-01            Action   

   ttm_consciousness_raising  ttm_dramatic_relief  \
0                   1.333333             2.666667   
0                   3.333333             2.666667   
0                   1.000000             2.000000   
0                   2.000000             2.000000   
0                   3.000000             4.666667   
0                   2.333333             1.333333   
0                   2.666667             2.666667   
0                   2.000000             2.666667   
0                   3.333333             2.666667   
0                   1.000000             2.333333   
0                   2.666667             2.000000   
0                   2.333333             3.333333   
2                   4.333333             2.666667   
0                   2.333333             2

Merge Fitbit with TTM

In [4]:
data = fitbit.merge(ttm[['id', 'stage']], on=['id'])
data

id       date  nightly_temperature  nremhr  \
0     621e2e8e67b776a24055b564 2021-05-24            34.137687  57.432   
1     621e2e8e67b776a24055b564 2021-05-25            33.794544  57.681   
2     621e2e8e67b776a24055b564 2021-05-26            34.611011  57.481   
3     621e2e8e67b776a24055b564 2021-05-27            34.408304  57.493   
4     621e2e8e67b776a24055b564 2021-05-28            34.178922  56.750   
...                        ...        ...                  ...     ...   
3642  621e339967b776a240e502de 2022-01-13            33.816660  51.760   
3643  621e339967b776a240e502de 2022-01-14            34.460723  51.351   
3644  621e339967b776a240e502de 2022-01-15            34.137341  50.504   
3645  621e339967b776a240e502de 2022-01-16            34.406884  51.533   
3646  621e339967b776a240e502de 2022-01-17            34.197504     NaN   

        rmssd  spo2  full_sleep_breathing_rate  stress_score  \
0      89.603   NaN                       14.8          78.0   
1      94.303   NaN                       15.8          80.0   
2     119.212   NaN                       14.6          84.0   
3     111.709   NaN                       14.8          82.0   
4     103.034   NaN                       15.2          81.0   
...       ...   ...                        ...           ...   
3642   47.281  96.7                       11.6          79.0   
3643   45.883  95.7                       12.0          79.0   
3644   49.779  95.7                       12.0          78.0   
3645   45.322  94.5                       11.6          73.0   
3646      NaN  95.7                       11.4          84.0   

      sleep_points_percentage  exertion_points_percentage  ...  Run  Spinning  \
0                    0.833333                       0.675  ...  0.0       0.0   
1                    0.833333                       0.725  ...  0.0       0.0   
2                    0.966667                       0.725  ...  0.0       0.0   
3                    0.933333                       0.725  ...  0.0       0.0   
4                    0.866667                       0.725  ...  0.0       0.0   
...                       ...                         ...  ...  ...       ...   
3642                 0.700000                       0.725  ...  1.0       0.0   
3643                 0.766667                       0.775  ...  NaN       NaN   
3644                 0.833333                       0.700  ...  0.0       0.0   
3645                 0.700000                       0.725  ...  0.0       0.0   
3646                 0.900000                       0.700  ...  NaN       NaN   

      Sport  Swim  Treadmill  Walk  Weights  Workout  Yoga/Pilates  \
0       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
1       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
2       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
3       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
4       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
...     ...   ...        ...   ...      ...      ...           ...   
3642    0.0   0.0        0.0   0.0      0.0      0.0           0.0   
3643    NaN   NaN        NaN   NaN      NaN      NaN           NaN   
3644    0.0   0.0        0.0   0.0      0.0      0.0           0.0   
3645    0.0   0.0        0.0   0.0      0.0      0.0           0.0   
3646    NaN   NaN        NaN   NaN      NaN      NaN           NaN   

            stage  
0     Maintenance  
1     Maintenance  
2     Maintenance  
3     Maintenance  
4     Maintenance  
...           ...  
3642  Maintenance  
3643  Maintenance  
3644  Maintenance  
3645  Maintenance  
3646  Maintenance  

[3647 rows x 65 columns]

In [5]:
data.to_pickle('./data/fitbit_ttm_to_model.pkl')

Load BREQ

In [6]:
breq = pd.read_pickle('./data/surveys/breq.pkl')
breq = breq.rename(columns={"user_id": "id", "submitdate": "date"})
breq["date"] = pd.to_datetime(pd.to_datetime(breq["date"]).dt.date)
breq.drop_duplicates(subset='id', keep='first', inplace=True)
breq = breq.drop(columns=['type'])
breq

id       date  breq_amotivation  \
0  621e2e8e67b776a24055b564 2021-05-31              1.00   
0  621e2eaf67b776a2406b14ac 2021-11-29              1.25   
0  621e2ed667b776a24085d8d1 2021-06-07              2.75   
0  621e2f3967b776a240c654db 2021-05-31              1.00   
0  621e2f6167b776a240e082a9 2021-06-01              1.00   
0  621e2f7a67b776a240f14425 2021-05-31              1.00   
0  621e2f9167b776a240011ccb 2021-11-28              1.00   
0  621e2fb367b776a24015accd 2021-07-26              1.25   
0  621e2fce67b776a240279baa 2021-05-31              1.00   
0  621e2ff067b776a2403eb737 2021-11-29              1.25   
0  621e300767b776a2404dc717 2022-01-17              1.00   
0  621e301367b776a24057738e 2021-06-01              1.00   
1  621e301e67b776a240608a72 2021-08-28              1.00   
0  621e30c867b776a240d4aa6c 2021-08-01              1.00   
0  621e30e267b776a240e5bf90 2021-11-23              1.75   
0  621e30e467b776a240e817c7 2021-08-01              1.00   
0  621e30f467b776a240f22944 2021-11-29              1.00   
0  621e310d67b776a24003096d 2021-11-22              1.00   
0  621e312a67b776a240164d59 2021-11-22              2.00   
0  621e314867b776a24029ebf9 2021-05-31              1.00   
0  621e323667b776a240f19134 2021-11-23              1.00   
0  621e324e67b776a2400191cb 2021-11-27              1.50   
0  621e328667b776a240281372 2021-05-31              1.00   
0  621e329067b776a2402ffad2 2021-05-31              1.50   
0  621e32af67b776a24045b4cf 2021-05-31              1.25   
0  621e32d067b776a2405b7d54 2021-12-09              2.50   
0  621e32d967b776a240627414 2021-05-31              1.25   
0  621e32e667b776a2406d2f1c 2021-11-23              1.25   
0  621e331067b776a24085dd3f 2021-11-23              1.00   
0  621e332267b776a24092a584 2021-06-02              1.00   
0  621e333567b776a240a0c217 2021-11-23              1.25   
0  621e333967b776a240a3cd06 2021-11-27              1.00   
0  621e335a67b776a240bb12ff 2021-05-31              1.00   
0  621e337667b776a240ce78ab 2021-06-07              1.25   
0  621e339967b776a240e502de 2022-01-20              1.00   
0  621e33b067b776a240f39e56 2021-06-08              1.00   
0  621e33cf67b776a240087de9 2021-11-24              1.00   
0  621e33ed67b776a2401cf5f7 2021-06-03              1.00   
0  621e341067b776a24037b105 2021-05-31              2.00   
0  621e345c67b776a2407146a8 2021-08-17              1.00   
0  621e346f67b776a24081744f 2022-01-24              1.50   
0  621e34db67b776a240c9c2be 2021-05-31              1.00   
0  621e34ec67b776a240d60873 2021-06-02              1.00   
0  621e34f767b776a240de4e1a 2021-07-26              3.50   
0  621e351a67b776a240f6204b 2021-07-26              1.00   
0  621e356967b776a24027bd9f 2021-11-24              1.00   
0  621e362467b776a2404ad513 2021-06-02              2.50   
0  621e366567b776a24076a727 2021-11-22              1.00   
0  621e367e67b776a24087d75d 2021-11-24              4.00   
0  621e36c267b776a240ba2756 2021-05-31              1.00   
0  621e36f967b776a240e5e7c9 2021-05-31              1.00   
0  621e375b67b776a240290cdc 2021-06-01              1.00   

   breq_external_regulation  breq_introjected_regulation  \
0                      1.00                     1.000000   
0                      1.00                     1.666667   
0                      2.00                     3.000000   
0                      2.00                     1.000000   
0                      1.25                     2.333333   
0                      1.25                     1.000000   
0                      1.00                     4.333333   
0                      1.75                     3.333333   
0                      2.50                     3.000000   
0                      2.25                     2.333333   
0                      2.25                     2.000000   
0                      1.50                     3.000000   
1                      1.50                     2.000000   
0   

Merge Fitbit with BREQ

In [8]:
data = fitbit.merge(breq[['id', 'breq_self_determination']], on=['id'])
data

id       date  nightly_temperature  nremhr  \
0     621e2e8e67b776a24055b564 2021-05-24            34.137687  57.432   
1     621e2e8e67b776a24055b564 2021-05-25            33.794544  57.681   
2     621e2e8e67b776a24055b564 2021-05-26            34.611011  57.481   
3     621e2e8e67b776a24055b564 2021-05-27            34.408304  57.493   
4     621e2e8e67b776a24055b564 2021-05-28            34.178922  56.750   
...                        ...        ...                  ...     ...   
3578  621e339967b776a240e502de 2022-01-13            33.816660  51.760   
3579  621e339967b776a240e502de 2022-01-14            34.460723  51.351   
3580  621e339967b776a240e502de 2022-01-15            34.137341  50.504   
3581  621e339967b776a240e502de 2022-01-16            34.406884  51.533   
3582  621e339967b776a240e502de 2022-01-17            34.197504     NaN   

        rmssd  spo2  full_sleep_breathing_rate  stress_score  \
0      89.603   NaN                       14.8          78.0   
1      94.303   NaN                       15.8          80.0   
2     119.212   NaN                       14.6          84.0   
3     111.709   NaN                       14.8          82.0   
4     103.034   NaN                       15.2          81.0   
...       ...   ...                        ...           ...   
3578   47.281  96.7                       11.6          79.0   
3579   45.883  95.7                       12.0          79.0   
3580   49.779  95.7                       12.0          78.0   
3581   45.322  94.5                       11.6          73.0   
3582      NaN  95.7                       11.4          84.0   

      sleep_points_percentage  exertion_points_percentage  ...  Run  Spinning  \
0                    0.833333                       0.675  ...  0.0       0.0   
1                    0.833333                       0.725  ...  0.0       0.0   
2                    0.966667                       0.725  ...  0.0       0.0   
3                    0.933333                       0.725  ...  0.0       0.0   
4                    0.866667                       0.725  ...  0.0       0.0   
...                       ...                         ...  ...  ...       ...   
3578                 0.700000                       0.725  ...  1.0       0.0   
3579                 0.766667                       0.775  ...  NaN       NaN   
3580                 0.833333                       0.700  ...  0.0       0.0   
3581                 0.700000                       0.725  ...  0.0       0.0   
3582                 0.900000                       0.700  ...  NaN       NaN   

      Sport  Swim  Treadmill  Walk  Weights  Workout  Yoga/Pilates  \
0       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
1       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
2       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
3       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
4       0.0   0.0        0.0   1.0      0.0      0.0           0.0   
...     ...   ...        ...   ...      ...      ...           ...   
3578    0.0   0.0        0.0   0.0      0.0      0.0           0.0   
3579    NaN   NaN        NaN   NaN      NaN      NaN           NaN   
3580    0.0   0.0        0.0   0.0      0.0      0.0           0.0   
3581    0.0   0.0        0.0   0.0      0.0      0.0           0.0   
3582    NaN   NaN        NaN   NaN      NaN      NaN           NaN   

      breq_self_determination  
0        intrinsic_regulation  
1        intrinsic_regulation  
2        intrinsic_regulation  
3        intrinsic_regulation  
4        intrinsic_regulation  
...                       ...  
3578    identified_regulation  
3579    identified_regulation  
3580    identified_regulation  
3581    identified_regulation  
3582    identified_regulation  

[3583 rows x 65 columns]

In [9]:
data.to_pickle('./data/fitbit_breq_to_model.pkl')

Load Personality

In [11]:
big5 = pd.read_pickle('./data/surveys/personality.pkl')
big5 = big5.rename(columns={"user_id": "id", "submitdate": "date"})
big5["date"] = pd.to_datetime(pd.to_datetime(big5["date"]).dt.date)
big5.drop_duplicates(subset='id', keep='first', inplace=True)
big5 = big5.drop(columns=['type'])
big5

id       date  extraversion  agreeableness  \
0   621e2e8e67b776a24055b564 2021-05-31          21.0           33.0   
1   621e2eaf67b776a2406b14ac 2021-11-29          32.0           45.0   
2   621e2ed667b776a24085d8d1 2021-06-07          40.0           43.0   
3   621e2f3967b776a240c654db 2021-05-31          25.0           34.0   
4   621e2f6167b776a240e082a9 2021-06-01          41.0           41.0   
5   621e2f7a67b776a240f14425 2021-05-31          29.0           38.0   
6   621e2f9167b776a240011ccb 2021-11-28          21.0           45.0   
7   621e2fb367b776a24015accd 2021-05-31          37.0           36.0   
8   621e2fce67b776a240279baa 2021-05-31          34.0           42.0   
9   621e2ff067b776a2403eb737 2021-11-29          33.0           44.0   
10  621e301367b776a24057738e 2021-06-01          26.0           36.0   
11  621e301e67b776a240608a72 2021-05-31          35.0           44.0   
12  621e30c867b776a240d4aa6c 2021-06-04          34.0           40.0   
13  621e30e267b776a240e5bf90 2021-11-23          38.0           42.0   
14  621e30e467b776a240e817c7 2021-06-06          34.0           20.0   
15  621e30f467b776a240f22944 2021-11-29          28.0           40.0   
16  621e310d67b776a24003096d 2021-12-03          43.0           48.0   
17  621e312a67b776a240164d59 2021-11-22          25.0           33.0   
18  621e314867b776a24029ebf9 2021-05-31          20.0           40.0   
19  621e323667b776a240f19134 2021-11-23          39.0           44.0   
20  621e324e67b776a2400191cb 2021-11-27          32.0           39.0   
21  621e328667b776a240281372 2021-05-31          37.0           39.0   
22  621e329067b776a2402ffad2 2021-05-31          29.0           47.0   
23  621e32af67b776a24045b4cf 2021-05-31          18.0           36.0   
24  621e32d067b776a2405b7d54 2021-12-09          21.0           36.0   
25  621e32d967b776a240627414 2021-05-31          34.0           34.0   
26  621e331067b776a24085dd3f 2021-11-23          19.0           40.0   
27  621e332267b776a24092a584 2021-06-02          22.0           32.0   
28  621e333567b776a240a0c217 2021-11-23          30.0           41.0   
29  621e333967b776a240a3cd06 2021-11-27          22.0           37.0   
30  621e335a67b776a240bb12ff 2021-05-31          21.0           29.0   
31  621e337667b776a240ce78ab 2021-06-07          22.0           30.0   
32  621e339967b776a240e502de 2021-11-22          35.0           45.0   
34  621e33b067b776a240f39e56 2021-06-08          45.0           49.0   
35  621e33cf67b776a240087de9 2021-11-24          16.0           31.0   
36  621e33ed67b776a2401cf5f7 2021-06-03          39.0           48.0   
37  621e341067b776a24037b105 2021-05-31          19.0           30.0   
38  621e346f67b776a24081744f 2021-11-24          35.0           44.0   
39  621e34db67b776a240c9c2be 2021-05-31          28.0           45.0   
40  621e34ec67b776a240d60873 2021-06-02          31.0           29.0   
41  621e34f767b776a240de4e1a 2021-06-07          40.0           42.0   
42  621e356967b776a24027bd9f 2021-11-24          26.0           38.0   
43  621e362467b776a2404ad513 2021-06-02          19.0           36.0   
45  621e366567b776a24076a727 2021-11-22          30.0           36.0   
46  621e367e67b776a24087d75d 2021-11-24          13.0           26.0   
47  621e36bb67b776a240b40d64 2021-06-02          41.0           29.0   
48  621e36c267b776a240ba2756 2021-05-31          39.0           36.0   
49  621e36dd67b776a240ce9a45 2021-07-21          33.0           34.0   
50  621e36f967b776a240e5e7c9 2021-05-31          42.0           42.0   
52  621e375b67b776a240290cdc 2021-06-01          34.0           46.0   

    conscientiousness  stability  intellect  gender  \
0                45.0       42.0       40.0    MALE   
1                30.0       18.0       41.0  FEMALE   
2                22.0       28.0       34.0  FEMALE   
3                30.0       39.0       37.0    MALE   
4                30.0       34.0       30.0  FEMALE   
5                43.0  

Merge Fitbit with Personality

In [13]:
data = fitbit.merge(big5[['id', 'ipip_extraversion_category', 'ipip_agreeableness_category', 'ipip_conscientiousness_category', 'ipip_stability_category', 'ipip_intellect_category']], on=['id'])
data

id       date  nightly_temperature  nremhr  \
0     621e2e8e67b776a24055b564 2021-05-24            34.137687  57.432   
1     621e2e8e67b776a24055b564 2021-05-25            33.794544  57.681   
2     621e2e8e67b776a24055b564 2021-05-26            34.611011  57.481   
3     621e2e8e67b776a24055b564 2021-05-27            34.408304  57.493   
4     621e2e8e67b776a24055b564 2021-05-28            34.178922  56.750   
...                        ...        ...                  ...     ...   
3450  621e339967b776a240e502de 2022-01-13            33.816660  51.760   
3451  621e339967b776a240e502de 2022-01-14            34.460723  51.351   
3452  621e339967b776a240e502de 2022-01-15            34.137341  50.504   
3453  621e339967b776a240e502de 2022-01-16            34.406884  51.533   
3454  621e339967b776a240e502de 2022-01-17            34.197504     NaN   

        rmssd  spo2  full_sleep_breathing_rate  stress_score  \
0      89.603   NaN                       14.8          78.0   
1      94.303   NaN                       15.8          80.0   
2     119.212   NaN                       14.6          84.0   
3     111.709   NaN                       14.8          82.0   
4     103.034   NaN                       15.2          81.0   
...       ...   ...                        ...           ...   
3450   47.281  96.7                       11.6          79.0   
3451   45.883  95.7                       12.0          79.0   
3452   49.779  95.7                       12.0          78.0   
3453   45.322  94.5                       11.6          73.0   
3454      NaN  95.7                       11.4          84.0   

      sleep_points_percentage  exertion_points_percentage  ...  Treadmill  \
0                    0.833333                       0.675  ...        0.0   
1                    0.833333                       0.725  ...        0.0   
2                    0.966667                       0.725  ...        0.0   
3                    0.933333                       0.725  ...        0.0   
4                    0.866667                       0.725  ...        0.0   
...                       ...                         ...  ...        ...   
3450                 0.700000                       0.725  ...        0.0   
3451                 0.766667                       0.775  ...        NaN   
3452                 0.833333                       0.700  ...        0.0   
3453                 0.700000                       0.725  ...        0.0   
3454                 0.900000                       0.700  ...        NaN   

      Walk  Weights  Workout  Yoga/Pilates  ipip_extraversion_category  \
0      1.0      0.0      0.0           0.0                         LOW   
1      1.0      0.0      0.0           0.0                         LOW   
2      1.0      0.0      0.0           0.0                         LOW   
3      1.0      0.0      0.0           0.0                         LOW   
4      1.0      0.0      0.0           0.0                         LOW   
...    ...      ...      ...           ...                         ...   
3450   0.0      0.0      0.0           0.0                     AVERAGE   
3451   NaN      NaN      NaN           NaN                     AVERAGE   
3452   0.0      0.0      0.0           0.0                     AVERAGE   
3453   0.0      0.0      0.0           0.0                     AVERAGE   
3454   NaN      NaN      NaN           NaN                     AVERAGE   

      ipip_agreeableness_category  ipip_conscientiousness_category  \
0                             LOW                             HIGH   
1                             LOW                             HIGH   
2                             LOW                             HIGH   
3                             LOW                             HIGH   
4                             LOW                             HIGH   
...                           ...                              ...   
3450                         HIGH                          AVERAGE

In [14]:
data.to_pickle('./data/fitbit_personality_to_model.pkl')

Find the users

In [3]:
users = list(fitbit['id'].unique())

Load PANAS

In [4]:
panas = pd.read_pickle('./data/surveys/panas.pkl')
panas = panas.rename(columns={"user_id": "id", "submitdate": "date"})
panas["date"] = pd.to_datetime(pd.to_datetime(panas["date"]).dt.date)
panas.drop_duplicates(subset=['id', 'date'], keep='first', inplace=True)
panas = panas.drop(columns=['type'])
panas

id       date  positive_affect_score  \
1   621e2e8e67b776a24055b564 2021-07-26                     37   
2   621e2e8e67b776a24055b564 2021-05-31                     38   
3   621e2e8e67b776a24055b564 2021-06-07                     37   
4   621e2e8e67b776a24055b564 2021-06-28                     31   
5   621e2e8e67b776a24055b564 2021-07-19                     37   
..                       ...        ...                    ...   
1   621e375367b776a24021e950 2022-01-02                     30   
0   621e375b67b776a240290cdc 2021-06-01                     27   
1   621e375b67b776a240290cdc 2021-06-08                     30   
2   621e375b67b776a240290cdc 2021-06-22                     30   
3   621e375b67b776a240290cdc 2021-07-06                     30   

    negative_affect_score  
1                      14  
2                      12  
3                      12  
4                      12  
5                      12  
..                    ...  
1                      33  
0                      29  
1                      30  
2                      30  
3                      29  

[268 rows x 4 columns]

In [5]:
fitbit_panas = preprocessing_functions.weekly_fitbit_frequency(panas, fitbit, users)
fitbit_panas

id       date positive_affect_score  \
0    621e2e8e67b776a24055b564 2021-07-26                    37   
1    621e2e8e67b776a24055b564 2021-05-31                    38   
2    621e2e8e67b776a24055b564 2021-06-07                    37   
3    621e2e8e67b776a24055b564 2021-06-28                    31   
4    621e2e8e67b776a24055b564 2021-07-19                    37   
..                        ...        ...                   ...   
263  621e339967b776a240e502de 2021-12-20                    44   
264  621e339967b776a240e502de 2021-12-27                    45   
265  621e339967b776a240e502de 2022-01-03                    44   
266  621e339967b776a240e502de 2022-01-12                    45   
267  621e339967b776a240e502de 2021-12-06                    41   

    negative_affect_score nightly_temperature  nremhr    rmssd  spo2  \
0                      14           34.515655  59.798   98.515   NaN   
1                      12           34.408304  57.481   94.303   NaN   
2                      12           34.476865   56.57  109.509   NaN   
3                      12           34.418045  53.006  107.388   NaN   
4                      12           34.254286   57.33  100.374   NaN   
..                    ...                 ...     ...      ...   ...   
263                    13           34.876472  50.683   47.293  96.0   
264                    11           35.107574  54.325   47.929  95.4   
265                    11           34.957428   52.81   47.801  94.9   
266                    13           34.073346  51.364   51.324  96.1   
267                    12           34.674633  51.192   49.232  96.2   

    full_sleep_breathing_rate stress_score  ... Martial Arts  Run Spinning  \
0                        15.4         74.0  ...          0.0  0.0      0.0   
1                        14.8         81.0  ...          0.0  0.0      0.0   
2                        14.4         83.0  ...          0.0  0.0      0.0   
3                        15.6         76.0  ...          0.0  0.0      0.0   
4                        15.0         75.0  ...          0.0  0.0      0.0   
..                        ...          ...  ...          ...  ...      ...   
263                      11.6         82.0  ...          0.0  0.0      0.0   
264                      12.2         87.0  ...          0.0  0.0      0.0   
265                      12.4         81.0  ...          0.0  0.0      0.0   
266                      12.2         84.0  ...          0.0  0.0      0.0   
267                      12.2         83.0  ...          0.0  0.0      0.0   

    Sport Swim Treadmill Walk Weights Workout Yoga/Pilates  
0     0.0  0.0       0.0  1.0     0.0     0.0          0.0  
1     0.0  0.0       0.0  1.0     0.0     0.0          0.0  
2     0.0  0.0       0.0  1.0     0.0     0.0          0.0  
3     0.0  0.0       0.0  1.0     0.0     0.0          0.0  
4     0.0  0.0       0.0  1.0     0.0     0.0          0.0  
..    ...  ...       ...  ...     ...     ...          ...  
263   0.0  0.0       0.0  0.0     0.0     0.0          0.0  
264   0.0  0.0       0.0  1.0     0.0     0.0          0.0  
265   0.0  0.0       0.0  1.0     0.0     0.0          0.0  
266   0.0  0.0       0.0  1.0     0.0     0.0          0.0  
267   0.0  0.0       0.0  1.0     0.0     0.0          0.0  

[268 rows x 66 columns]

In [6]:
fitbit_panas.to_pickle('./data/fitbit_panas_to_model.pkl')

Load STAI

In [7]:
stai = pd.read_pickle('./data/surveys/stai.pkl')
stai = stai.rename(columns={"user_id": "id", "submitdate": "date"})
stai["date"] = pd.to_datetime(pd.to_datetime(stai["date"]).dt.date)
stai.drop_duplicates(subset=['id', 'date'], keep='first', inplace=True)
stai = stai.drop(columns=['type'])
stai

id       date  stai_stress stai_stress_category
0   621e2e8e67b776a24055b564 2021-05-31         45.0        Below average
1   621e2e8e67b776a24055b564 2021-06-07         46.0              Average
3   621e2e8e67b776a24055b564 2021-07-19         46.0              Average
4   621e2e8e67b776a24055b564 2021-07-26         46.0              Average
5   621e2e8e67b776a24055b564 2021-06-28         54.0        Above average
..                       ...        ...          ...                  ...
0   621e375b67b776a240290cdc 2021-06-08         60.0        Above average
1   621e375b67b776a240290cdc 2021-07-06         60.0        Above average
2   621e375b67b776a240290cdc 2021-06-01         59.0        Above average
3   621e375b67b776a240290cdc 2021-06-22         60.0        Above average
4   621e375b67b776a240290cdc 2021-07-12         51.0        Above average

[279 rows x 4 columns]

In [8]:
fitbit_stai = preprocessing_functions.weekly_fitbit_frequency(stai, fitbit, users)
fitbit_stai

id       date stai_stress stai_stress_category  \
0    621e2e8e67b776a24055b564 2021-05-31        45.0        Below average   
1    621e2e8e67b776a24055b564 2021-06-07        46.0              Average   
2    621e2e8e67b776a24055b564 2021-07-19        46.0              Average   
3    621e2e8e67b776a24055b564 2021-07-26        46.0              Average   
4    621e2e8e67b776a24055b564 2021-06-28        54.0        Above average   
..                        ...        ...         ...                  ...   
274  621e339967b776a240e502de 2021-11-22        49.0              Average   
275  621e339967b776a240e502de 2021-12-06        51.0        Above average   
276  621e339967b776a240e502de 2021-12-14        45.0        Below average   
277  621e339967b776a240e502de 2021-12-20        48.0              Average   
278  621e339967b776a240e502de 2022-01-12        49.0              Average   

    nightly_temperature  nremhr    rmssd  spo2 full_sleep_breathing_rate  \
0             34.408304  57.481   94.303   NaN                      14.8   
1             34.476865   56.57  109.509   NaN                      14.4   
2             34.254286   57.33  100.374   NaN                      15.0   
3             34.515655  59.798   98.515   NaN                      15.4   
4             34.418045  53.006  107.388   NaN                      15.6   
..                  ...     ...      ...   ...                       ...   
274                 NaN     NaN      NaN   NaN                       NaN   
275           34.674633  51.192   49.232  96.2                      12.2   
276           34.303759  46.311   56.468  96.5                      11.0   
277           34.876472  50.683   47.293  96.0                      11.6   
278           34.073346  51.364   51.324  96.1                      12.2   

    stress_score  ... Martial Arts  Run Spinning Sport Swim Treadmill Walk  \
0           81.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  1.0   
1           83.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  1.0   
2           75.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  1.0   
3           74.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  1.0   
4           76.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  1.0   
..           ...  ...          ...  ...      ...   ...  ...       ...  ...   
274          NaN  ...          NaN  NaN      NaN   NaN  NaN       NaN  NaN   
275         83.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  1.0   
276         87.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  1.0   
277         82.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  0.0   
278         84.0  ...          0.0  0.0      0.0   0.0  0.0       0.0  1.0   

    Weights Workout Yoga/Pilates  
0       0.0     0.0          0.0  
1       0.0     0.0          0.0  
2       0.0     0.0          0.0  
3       0.0     0.0          0.0  
4       0.0     0.0          0.0  
..      ...     ...          ...  
274     NaN     NaN          NaN  
275     0.0     0.0          0.0  
276     0.0     0.0          0.0  
277     0.0     0.0          0.0  
278     0.0     0.0          0.0  

[279 rows x 66 columns]

In [9]:
fitbit_stai.to_pickle('./data/fitbit_stai_to_model.pkl')